# Mount to Drive


In [2]:
# Setup 1. Connect Google Drive
from google.colab import drive
drive.mount('/content/gdrive',force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly&response_type=code

Enter your authorization code:
4/3wG9kOQrx2L8qt0993ZaiVprovH-C-Bo__DkxfdfMtVKakZyKN6Aklk
Mounted at /content/gdrive


# Variables

In [3]:
import torch
import os
from skimage import io
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader, ConcatDataset
import numpy as np
import torchvision.transforms as transforms
import torch.nn.functional as F
#--------------------------------------------------------------
import torch.nn as nn
import torchvision.models as models
import torch.optim as optim

USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device("cuda" if USE_CUDA else "cpu")


#model hyperparameters
model_output_size=4
BATCH_SIZE = 8
learning_rate=0.0001
train_epoch=3 #10

#EPOCH = 1000 #
transforms = transforms.Compose([transforms.ToTensor(), transforms.Normalize(mean=[0.485,0.456,0.406],
                                                                             std=[0.229,0.224,0.225])])



# DATA LOAD

In [4]:
# Setup 2. extract zip: Please download dataset [train.zip, test100/404.zip, validation.zip]
import zipfile
file_path = '/content/gdrive/My Drive/DL_repos/competition'
!cp "{file_path}" "./"

cp: -r not specified; omitting directory '/content/gdrive/My Drive/DL_repos/competition'


In [9]:
# Setup 3. unzip and check
!unzip -q -o '/content/gdrive/My Drive/DL_repos/competition/train.zip' -d '/content/gdrive/My Drive/DL_repos/competition/train/'
!unzip -q -o '/content/gdrive/My Drive/DL_repos/competition/test100.zip' -d '/content/gdrive/My Drive/DL_repos/competition/test100/'
!unzip -q -o '/content/gdrive/My Drive/DL_repos/competition/validation.zip' -d '/content/gdrive/My Drive/DL_repos/competition/validation/'

train_cheetah_root=file_path+"/train/cheetah_train_resized"
train_jaguar_root=file_path+"/train/jaguar_train_resized"
train_tiger_root=file_path+"/train/tiger_train_resized"
train_hyena_root=file_path+"/train/hyena_train_resized"

validation_cheetah_root=file_path+"/validation/cheetah_validation_resized"
validation_jaguar_root=file_path+"/validation/jaguar_validation_resized"
validation_tiger_root=file_path+"/validation/tiger_validation_resized"
validation_hyena_root=file_path+"/validation/hyena_validation_resized"

test100_root=file_path+"/test100"

print(len(os.listdir(train_cheetah_root)))
print(len(os.listdir(validation_cheetah_root)))
print(len(os.listdir(test100_root)))

900
100
100


In [10]:
class MyDataset(Dataset):
  def __init__(self, image_dir, label, transforms=None):
    self.image_dir = image_dir
    self.label = label
    self.image_list = os.listdir(self.image_dir)
    self.transforms = transforms
  
  def __len__(self):
    return len(self.image_list)
  
  def __getitem__(self,idx):
    # if torch.is_tensor(idx):
    #   idx = idx.tolist()

    image_name = os.path.join(self.image_dir, self.image_list[idx])
    image = io.imread(image_name)

    ### transform
    image = transforms(image)

    return (image,self.label)

root = '/content/gdrive/My Drive/DL_repos/competition' #'/kaggle/input/swdl2020': root project repository
#cheetah : 0 , jaguar : 1, tiger : 2, hyena : 3

cheetah_train = MyDataset(root+"/train/cheetah_train_resized",0,transforms)
jaguar_train = MyDataset(root+"/train/jaguar_train_resized",1,transforms)
tiger_train = MyDataset(root+"/train/tiger_train_resized",2,transforms)
hyena_train = MyDataset(root+"/train/hyena_train_resized",3,transforms)
train_set = ConcatDataset([cheetah_train, jaguar_train, tiger_train, hyena_train])
print("Number of Training set images : ", len(train_set))

cheetah_val = MyDataset(root+"/validation/cheetah_validation_resized",0, transforms)
jaguar_val = MyDataset(root+"/validation/jaguar_validation_resized",1, transforms)
tiger_val = MyDataset(root+"/validation/tiger_validation_resized",2, transforms)
hyena_val = MyDataset(root+"/validation/hyena_validation_resized",3,transforms)
val_set = ConcatDataset([cheetah_val, jaguar_val, tiger_val, hyena_val])
print("Numver of Validation set images : ", len(val_set))

train_loader = DataLoader(train_set, batch_size=BATCH_SIZE, shuffle = True) #params: train_set, 16, True
val_loader = DataLoader(val_set, batch_size=BATCH_SIZE, shuffle=True)

Number of Training set images :  3600
Numver of Validation set images :  400


# MODEL

In [11]:
##### YOUR MODEL#####
class myResnet18(nn.Module):
  def __init__(self):
    super(myResnet18, self).__init__()

    #Define Convolution Operation(attributes)
    self.resnet = models.resnet18() 
    self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    self.fc = nn.Linear(in_features=1000, out_features=model_output_size, bias=True) #nn.Linear()

  def forward(self, x):
    input=x
    #x = x.unsqueeze(0) # [3, 400, 400] -> [1, 3, 400, 400]
    x = self.resnet(x).to(DEVICE) # [1, 3, 400, 400] -> [1, 1000]
    x = x.view(-1, 1000) # INPUT [1, 64, 16, 16] -> OUTPUT [1, 64*16*16]  #input텐서를 지정된 output텐서로 펴주는 과정. [1,16384]로.
    x = F.softmax(self.fc(x),dim=1)#torch.sigmoid(self.fc(x)) # INPUT [1, 64*16*16] -> OUTPUT [1, 1]
    return x

resnetModel = myResnet18()
loss_func=nn.CrossEntropyLoss()


x = torch.rand(3, 400, 400)
print('1- ',x.shape)
x=x.unsqueeze(0)
x=resnetModel.resnet(x)
print('2- ',x.shape)
x=x.view(-1,1000)
print('3- ',x.shape)
x=F.softmax(resnetModel.fc(x),dim=1)
print('4- ',x.shape)


1-  torch.Size([3, 400, 400])
2-  torch.Size([1, 1000])
3-  torch.Size([1, 1000])
4-  torch.Size([1, 4])


# TRAIN

In [12]:
from tqdm.notebook import tqdm # for visualize training step. iterative한 것의 진행도를 보여줌. 트레이닝 정도를 시각화. 

def train(model, train_loader, optimizer, epoch):
  model.train() #training mode
  #for batch_idx, (image, target) in enumerate(train_loader): #target=label
  for batch_idx, (image, target) in tqdm(enumerate(train_loader)):
    data, target = image.to(DEVICE), target.to(DEVICE)
    optimizer.zero_grad()
    output = model(data).to(DEVICE) #cuda()
    output=output.squeeze(1)
    loss = loss_func(output,target) #nn.CrossEntropyLoss(output,target) 
    loss.backward()
    optimizer.step()

    if batch_idx % 150 == 0 :
      print('Train Epoch : {} [{}/{} ({:.0f})%]\tLoss: {:.6f}'
      .format(epoch, batch_idx*len(image),len(train_loader.dataset), 100.*batch_idx / len(train_loader), loss.item()))

def evaluate(model, test_loader):
  model.eval() #validation mode
  test_loss =0
  correct =0
  with torch.no_grad():
    for (image, target) in tqdm(test_loader):
      image, label = image.to(DEVICE), target.to(DEVICE)
      output = model(image).to(DEVICE)

      test_loss += F.cross_entropy(output, label, reduction='sum').item()
      pred = output.max(1, keepdim=True)[1]
      correct+= pred.eq(label.view_as(pred)).sum().item()
  
  test_loss /= len(test_loader.dataset)
  test_accuracy = 100. * correct / len(test_loader.dataset)
  return test_loss, test_accuracy

# EXECUTE

In [13]:
#gpu model

model=myResnet18().to(DEVICE)#
criterion=nn.CrossEntropyLoss() #nn.BCELoss() #Binary Cross Entropy Loss
optimizer= torch.optim.Adam(model.parameters(), lr=learning_rate) #params: net.params(),0.001,


#????
train_info=[]
test_info=[]
save_path='./ClassificationNetwork/'


#training epoches
for epoch in range(1, train_epoch+1):
  print('#---------------------------------------------')
  print('train epoch: {}'.format(epoch))
  train(model, train_loader, optimizer, epoch)
  #params: model(myCNN), DataLoader(train_set, 16, True), SGD(model.parameters(),lr=0.001,momentum=0.9) , EPOCH(1000)
  print('validation epoch: {}'.format(epoch))
  test_loss, test_accuracy = evaluate(model, val_loader)
  print('[epoch {} Result] Test Loss : {:.4f}, Accuracy : {:.4f}%'.format(epoch, test_loss, test_accuracy))

#---------------------------------------------
train epoch: 1


Train Epoch : 1 [0/3600 (0)%]	Loss: 1.405159
Train Epoch : 1 [1200/3600 (33)%]	Loss: 1.177507
Train Epoch : 1 [2400/3600 (67)%]	Loss: 1.017727

validation epoch: 1



[epoch 1 Result] Test Loss : 1.0712, Accuracy : 66.7500%
#---------------------------------------------
train epoch: 2


Train Epoch : 2 [0/3600 (0)%]	Loss: 1.012108
Train Epoch : 2 [1200/3600 (33)%]	Loss: 0.975389
Train Epoch : 2 [2400/3600 (67)%]	Loss: 0.991266

validation epoch: 2



[epoch 2 Result] Test Loss : 0.9532, Accuracy : 79.0000%
#---------------------------------------------
train epoch: 3


Train Epoch : 3 [0/3600 (0)%]	Loss: 1.042063
Train Epoch : 3 [1200/3600 (33)%]	Loss: 1.067788
Train Epoch : 3 [2400/3600 (67)%]	Loss: 1.044620

validation epoch: 3



[epoch 3 Result] Test Loss : 0.9837, Accuracy : 76.0000%


# TEST LOADER

In [15]:
class TestDataset(Dataset):
  def __init__(self, image_dir, transforms=None):
    self.image_dir = image_dir
    self.image_list = os.listdir(self.image_dir)
    self.transforms = transforms
  
  def __len__(self):
    return len(self.image_list)
  
  def __getitem__(self,idx):
    # if torch.is_tensor(idx):
    #   idx = idx.tolist()

    image_name = os.path.join(self.image_dir, self.image_list[idx])
    image = io.imread(image_name)

    ### transform
    image = transforms(image)

    return (image,self.image_list[idx].split('.')[0])


test_set = TestDataset(root+"/test100", transforms) #test100
test_loader = DataLoader(test_set)

# PREDICTION

In [16]:
import pandas as pd

#cheetah : 0 , jaguar : 1, tiger : 2, hyena : 3
map = ['cheetah','jaguar','tiger','hyena']

model.eval()
df = pd.DataFrame(columns=['id','category'])
with torch.no_grad():
    #for (image, image_name) in test_loader:
    for (image, image_name) in tqdm(test_loader):
        image = image.to(DEVICE)
        output = model(image).to(DEVICE)
        pred = output.max(1, keepdim=True)[1]
        print('iter:')
        print('  1- ',image.shape)
        print('  2- ',image_name)
        print('  3- ',output.shape)
        print('  4- ',pred)
        
        df = df.append(pd.DataFrame([[image_name[0], map[pred.squeeze().tolist()]]], columns=['id','category']))
df #print df

iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('10083',)
  3-  torch.Size([1, 4])
  4-  tensor([[0]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('10295',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('10348',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('1062',)
  3-  torch.Size([1, 4])
  4-  tensor([[1]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('10714',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('1076',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('1087',)
  3-  torch.Size([1, 4])
  4-  tensor([[1]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  2-  ('11045',)
  3-  torch.Size([1, 4])
  4-  tensor([[3]], device='cuda:0')
iter:
  1-  torch.Size([1, 3, 400, 400])
  

,id,category
0,10083,cheetah
0,10295,hyena
0,10348,hyena
0,1062,jaguar
0,10714,hyena
...,...,...
0,8986,cheetah
0,9042,cheetah
0,938,hyena
0,9682,hyena


# SAVE CSV FILE

In [18]:
df.to_csv(root+'/result100.csv', index=False)
#df.to_csv(root+'/result.csv', index=False)